In [ ]:
#Importacion de librerias utiles

#Libreria para analisis de datos 
#en este proyecto para leer y grabar
#archivos en csv
import pandas

#Librerias cientificas scipy, numpy, matplotlib
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

#Librerias de incluidas en python time, math
import time
import math

#Lectura de archivos xml
from lxml import etree

#Expresiones regulares
import re

#Pyfreeling
from pyfreeling import Analyzer
analyzer = Analyzer(config='/usr/share/freeling/config/es.cfg', lang='es')

#Para implementar modelos de aprendizajes
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
vectorizer = CountVectorizer(stop_words='english')



# Preprocesamiento

# Eliminar comentarios con menos de tres votos

In [ ]:
#Se carga el corpus de humor
corpus = pandas.read_csv("corpus_humor_training.csv",encoding='utf-8')

#Listas para salvar los tweets filtrados
id_f = []
text_f = []
account_id_f=[]
nh_f=[]
sh1_f=[]
sh2_f=[]
sh3_f=[]
sh4_f=[]
sh5_f=[]

#Listas del corpus
vec_id=corpus['id'][:]
vec_text=corpus['text'][:]
vec_account_id=corpus['account_id'][:]
vec_no_humor =corpus['n'][:]
vec_e1 =corpus['1'][:]
vec_e2 =corpus['2'][:]
vec_e3 =corpus['3'][:]
vec_e4 =corpus['4'][:]
vec_e5 =corpus['5'][:]

#Para cada tweet se calcula si tiene al menos tres votos, 
#en caso afirmativo se guarda su informacion, en caso
#negativo se descarta el tweet
for i in range(len(corpus)):
    if vec_no_humor[i] + vec_e1[i] + vec_e2[i] + vec_e3[i] + vec_e4[i] + vec_e5[i] >= 3:
        id_f.append(vec_id[i])
        text_f.append(vec_text[i])
        account_id_f.append(vec_account_id[i])
        nh_f.append(vec_no_humor[i])
        sh1_f.append(vec_e1[i])
        sh2_f.append(vec_e2[i])
        sh3_f.append(vec_e3[i])
        sh4_f.append(vec_e4[i])
        sh5_f.append(vec_e5[i])
        
#El primer filtro aplicado es guardado en el archivo corpus_filtro1.csv
d = {'id' : id_f,
    'text' : text_f,
    'account_id': account_id_f,
    'n':nh_f, 
    '1':sh1_f,
    '2':sh2_f,
    '3':sh3_f,
    '4':sh4_f,
    '5':sh5_f
    }
df = pandas.DataFrame(d, columns = ['id', 'text', 'account_id', 'n', '1', '2', '3', '4', '5'])
df.to_csv('corpus_filtro1.csv')

## Eliminar Hashtags

In [ ]:
text_f = []

#Se carga la instancia de tweets salvada en el bloque de codigo anterior
corpus_filtro1 = pandas.read_csv("corpus_filtro1.csv",encoding='utf-8')

#Generacion de patron de Hashtag
pattern_hashtag = re.compile(r'#.+?\b')


#Se sustituye el Hashtag por el string vacio, los tweets que
#van pasando por este procesamiento se van guardando en el arreglo
#text_f
for i in range(len(corpus_filtro1)):
    text_f.append(re.sub(pattern_hashtag, "", corpus_filtro1['text'][i]))
#El segundo filtro aplicado es guardado en el archivo corpus_filtro2.csv
d = {'id' : corpus_filtro1['id'][:],
    'text' : text_f,
    'account_id': corpus_filtro1['account_id'][:],
    'n':corpus_filtro1['n'][:], 
    '1':corpus_filtro1['1'][:],
    '2':corpus_filtro1['2'][:],
    '3':corpus_filtro1['3'][:],
    '4':corpus_filtro1['4'][:],
    '5':corpus_filtro1['5'][:]
    }
df = pandas.DataFrame(d, columns = ['id', 'text', 'account_id', 'n', '1', '2', '3', '4', '5'])
df.to_csv('corpus_filtro2.csv')


# Agregar la categoria humor o no humor a los documentos

El criterio de humor o no humor esta basado en el criterio presentado en la letra del laboratorio, en donde se considerará humorístico si la mitad o más de los anotadores lo calificaron con una o más estrellas y no humorístico en caso contrario.

In [ ]:
#Se carga la instancia de tweets salvada en el bloque de codigo anterior
corpus_filtro2 = pandas.read_csv("corpus_filtro2.csv",encoding='utf-8')

#Listas para salvar los tweets filtrados
text = []
category=[]
h1=[]
h2=[]
h3=[]
h4=[]
h5=[]

#Listas del corpus
vec_text=corpus_filtro2['text'][:]
vec_no_humor =corpus_filtro2['n'][:]
vec_e1 =corpus_filtro2['1'][:]
vec_e2 =corpus_filtro2['2'][:]
vec_e3 =corpus_filtro2['3'][:]
vec_e4 =corpus_filtro2['4'][:]
vec_e5 =corpus_filtro2['5'][:]

#Si hay igual o mas votos de humor que de no humor la categoria tiene el valor 1
#en caso contrario tiene el valor 0
for i in range(len(corpus_filtro2)):
    if  vec_e1[i] + vec_e2[i] + vec_e3[i] + vec_e4[i] + vec_e5[i] >= vec_no_humor[i]:
        category.append(1)
    else:
        category.append(0)
    text.append(vec_text[i])
    h1.append(vec_e1[i])
    h2.append(vec_e2[i])
    h3.append(vec_e3[i])
    h4.append(vec_e4[i])
    h5.append(vec_e5[i])
#El tercer procesamiento aplicado sobre los tweets es guardado en el archivo corpus_filtro.3
d = {'text' : text,
    'category': category,
     '1':h1,
     '2':h2,
     '3':h3,
     '4':h4,
     '5':h5
    }
df = pandas.DataFrame(d, columns = ['text','category','1','2','3','4','5'])
df.to_csv('corpus_filtro3.csv')

## Agregar información de POS-tag 

In [ ]:
#Se carga la instancia de tweets salvada en el bloque de codigo anterior
corpus_filtro3 = pandas.read_csv("corpus_filtro3.csv",encoding='utf-8')
text_list=corpus_filtro3['text'][:]

#Se genera el patron para identificar las POS de las palabras
pattern_pos = re.compile(r'pos="(.*?)"')

#Variables para datos estadisticos
num_sentences=len(corpus_filtro3)
numsentence=1

#Variable auxiliar para adherir las POS a los tweets
pos=0

#Para cada documento en el corpus de tweets que va siendo procesado
#utilizando la herramienta freeling se analizan las categorias gramaticales
#de todas las palabras que aparecen en cada tweet y esta informacion es
#adherida al texto del tweet
for d in text_list:
    if(type(d) == str):
        xml = analyzer.run(d.encode(), 'flush')
        print(numsentence, " ", math.floor( numsentence/num_sentences*100),"%", end="\r")
        for sentence in xml:        
            for token in sentence:
                token_byte=etree.tostring(token)
                m = re.search(pattern_pos, token_byte.decode())
                if m is not None:
                    text_list[ pos] = text_list[pos] + " " + m.group(1)
    pos+=1
    numsentence+=1

#El procesamiento actual del corpus es guardado en el documento corpus_filtro4.csv
d = {'text' : text_list,
    'category': corpus_filtro3['category'][:],
     '1':corpus_filtro3['1'][:],
     '2':corpus_filtro3['2'][:],
     '3':corpus_filtro3['3'][:],
     '4':corpus_filtro3['4'][:],
     '5':corpus_filtro3['5'][:]
    }
df = pandas.DataFrame(d, columns = ['text', 'category', '1','2','3','4','5'])
df.to_csv('corpus_filtro4.csv')

# Separación de datos de entrenamiento y de desarrollo

In [ ]:
corpus_filtro4 = pandas.read_csv("corpus_filtro4.csv",encoding='utf-8')
num_dev_tweets=math.floor(len(corpus_filtro4)*20/100)
#Se generan posiciones en el arreglo tweets al azar que representen el 20% del corpus actual
random_samples=np.random.randint(0, len(corpus_filtro4) - 1, size=num_dev_tweets)

#Listas para entrenamiento
text_train = []
category_train=[]

#Listas para desarrollo
text_dev = []
category_dev=[]

#Listas con datos del corpus
vec_text=corpus_filtro4['text'][:]
vec_category=corpus_filtro4['category'][:]

#Se separa el 20% de corpus para desarrollo del 80% para entrenamiento
for i in range( len(corpus_filtro4)):
    if(np.any(random_samples[:] == i)):
        text_dev.append(vec_text[i])
        category_dev.append(vec_category[i])
    else:
        text_train.append(vec_text[i])
        category_train.append(vec_category[i])


#Se guardan las instancias corpus de desarrollo en corpus_filtro5_devset.csv
#y corpus de entrenamiento en corpus_filtro5_trainingset.csv
d = {'text' : text_train,
    'category': category_train
    }
df = pandas.DataFrame(d, columns = ['text', 'category'])
df.to_csv('corpus_filtro5_trainingset.csv')


d = {'text' : text_dev,
    'category': category_dev
    }
df = pandas.DataFrame(d, columns = ['text', 'category'])
df.to_csv('corpus_filtro5_devset.csv')

## Calculo de mediana, para segundo clasificador

In [ ]:
#Se carga la instancia salvada del corpus que se encuentra en el archivo
#corpus_filtro4.csv
corpus_filtro4 = pandas.read_csv("corpus_filtro4.csv",encoding='utf-8')

text_list=corpus_filtro4['text'][:]
category_list=corpus_filtro4['category'][:]

h1_list=corpus_filtro4['1'][:]
h2_list=corpus_filtro4['2'][:]
h3_list=corpus_filtro4['3'][:]
h4_list=corpus_filtro4['4'][:]
h5_list=corpus_filtro4['5'][:]

medians=[]
pos=0

#Caclulo de la mediana, en values se expanden los votos de cada estrella
#de la siguiente manera si #cant_3Estrellas_en_tweet = 4, entonces se guardan
#en values 4 treses 3,3,3,3 para permitir el calculo de la mediana
#Por ultimo en la estructura medians se guarda el valor discreto (0,1,2,3,4 o 5) de la mediana
#si la cantidad de votos es par se toma el voto del medio con valor mas grande
for i in range( len(corpus_filtro4)):
    values = []
    for j in range( h1_list[i]):
        values.append(1)
    for j in range( h2_list[i] ):
        values.append(2)
    for j in range( h3_list[i] ):
        values.append(3)
    for j in range( h4_list[i] ):
        values.append(4)
    for j in range( h5_list[i] ):
        values.append(5)
    if len(values) == 0:
        values.append(0)  
    mediana=np.median(values)
    if( len(values) % 2 == 1 ):
        medians.append(math.floor(mediana))
    else:
        for i in values:
            if( i >= mediana):
                medians.append(i)
                break
    pos+=1

#El nuevo procesamiento que suma la informacion de la mediana se guarda
#en el archivo corpus_filtro4median.csv
d = {'text' : corpus_filtro4['text'][:],
    'category' : corpus_filtro4['category'][:],
    'median': medians,
    '1':corpus_filtro4['1'][:],
    '2':corpus_filtro4['2'][:],
    '3':corpus_filtro4['3'][:],
    '4':corpus_filtro4['4'][:],
    '5':corpus_filtro4['5'][:]
    }
df = pandas.DataFrame(d, columns = ['text', 'category', 'median', '1', '2', '3', '4', '5'])
df.to_csv('corpus_filtro4median.csv')

## Separacion de datos de entrenamiento y desarrollo, para segundo clasificador 

In [28]:
import math
import numpy as np

#Se carga la instancia salvada en el bloque de codigo anterior
corpus_filtro4median = pandas.read_csv("corpus_filtro4median.csv",encoding='utf-8')

num_dev_tweets=math.floor(len(corpus_filtro4median)*20/100)
random_samples=np.random.randint(0, len(corpus_filtro4median) - 1, size=num_dev_tweets)

#Listas para entrenamiento
text_train = []
median_train=[]
category_train=[]

#Listas para desarrollo
text_dev = []
median_dev=[]
category_dev=[]

#Listas con datos del corpus
vec_text=corpus_filtro4median['text'][:]
vec_median=corpus_filtro4median['median'][:]
vec_category=corpus_filtro4median['category'][:]


for i in range( len(corpus_filtro4median)):
    if(np.any(random_samples[:] == i)):
        text_dev.append(vec_text[i])
        median_dev.append(vec_median[i])
        category_dev.append(vec_category[i])
        
    else:
        text_train.append(vec_text[i])
        median_train.append(vec_median[i])
        category_train.append(vec_category[i])

d = {'text' : text_train,
     'median' : median_train,
    'category': category_train
    }
df = pandas.DataFrame(d, columns = ['text', 'median','category'])
df.to_csv('corpus_filtro5_trainingset_c2.csv')

d = {'text' : text_dev,
     'median': median_dev,
    'category': category_dev
    }
df = pandas.DataFrame(d, columns = ['text', 'median', 'category'])
df.to_csv('corpus_filtro5_devset_c2.csv')

## Preprocesamiento, clasificador 3

In [ ]:
corpus_filtro4median = pandas.read_csv("corpus_filtro4median.csv",encoding='utf-8')
text_list=corpus_filtro4median['text'][:]
category_list=[]
median_list=corpus_filtro4median['median'][:]
h1_list=corpus_filtro4median['1'][:]
h2_list=corpus_filtro4median['2'][:]
h3_list=corpus_filtro4median['3'][:]
h4_list=corpus_filtro4median['4'][:]
h5_list=corpus_filtro4median['5'][:]
pos=0
for m in median_list:
    if(m<1):
        category_list.append(0)
    else:
        category_list.append(1)
    pos+=1
d = {'text' : text_list,
    'category': category_list,
     'median': median_list
    }
df = pandas.DataFrame(d, columns = ['text','category','median'])
df.to_csv('corpus_filtro4median_category.csv')


## Separación datos de entrenamiento y desarrollo, clasificador 3

In [ ]:
corpus_filtro4median_category = pandas.read_csv("corpus_filtro4median_category.csv",encoding='utf-8')
text=corpus_filtro4median_category['text'][:]
category=corpus_filtro4median_category['category'][:]
median=corpus_filtro4median_category['median'][:]

num_dev_tweets=math.floor(len(corpus_filtro4median_category)*20/100)
random_samples=np.random.randint(0, len(corpus_filtro4median_category) - 1, size=num_dev_tweets)

print(num_dev_tweets)
print(random_samples)

#Listas para entrenamiento
text_train = []
category_train=[]

#Listas para desarrollo
text_dev = []
category_dev=[]

#Listas con datos del corpus
vec_text=corpus_filtro4median_category['text'][:]
vec_category=corpus_filtro4median_category['category'][:]

for i in range( len(corpus_filtro4median_category)):
    if(np.any(random_samples[:] == i)):
        text_dev.append(vec_text[i])
        category_dev.append(vec_category[i])
    else:
        text_train.append(vec_text[i])
        category_train.append(vec_category[i])
d = {'text' : text_train,
    'category': category_train
    }
df = pandas.DataFrame(d, columns = ['text', 'category'])
df.to_csv('corpus_filtro5_trainingset_c3.csv')

d = {'text' : text_dev,
    'category': category_dev
    }
df = pandas.DataFrame(d, columns = ['text', 'category'])
df.to_csv('corpus_filtro5_devset_c3.csv')

In [145]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
import pandas

class Clasificador1:

    def __init__(self):
        self.vectorizer = CountVectorizer(stop_words='english')
        self.__clasiffier_1_train()
    
    def __clasiffier_1_train(self):
        corpus_filtro5_trainingset = pandas.read_csv("corpus_filtro5_trainingset.csv",encoding='utf-8')
        text_list=corpus_filtro5_trainingset['text'][:]
        category_list=corpus_filtro5_trainingset['category'][:]
        pos=0
        for t in text_list:
            if type(t) != str:
                text_list[pos]="NaN"
            pos+=1
        self.train_features = self.vectorizer.fit_transform(text_list)

        self.nb_1 = MultinomialNB()
        self.nb_1.fit(self.train_features , category_list)
    def predict_clasiffier_1(self, path):       
        corpus_filtro5_devset = pandas.read_csv(path,encoding='utf-8')
        text_devlist=corpus_filtro5_devset['text'][:]
        category_devlist=corpus_filtro5_devset['category'][:]
        pos=0
        for t in text_devlist:
            if type(t) != str:
                text_devlist[pos]="NaN"
            pos+=1
        self.test_features = self.vectorizer.transform(text_devlist)
        return self.nb_1.predict(self.test_features)

class Clasificador2:
    
    def __init__(self):
        self.vectorizer = CountVectorizer(stop_words='english')
        self.__clasiffier_2_train()
        self.__median_predictor_train()

    
    def __median_predictor_train(self):
        corpus_filtro5 = pandas.read_csv("corpus_filtro5_trainingset_c2.csv",encoding='utf-8')
        text_list=corpus_filtro5['text'][:]
        median_list=corpus_filtro5['median'][:]
        self.category_list_c1=[]

        pos=0
        for t in text_list:
            if type(t) != str:
                text_list[pos]="NaN"
            pos+=1

        self.train_features_c1 = vectorizer.fit_transform(text_list)

        for i in median_list:
            if(i == 1):
                self.category_list_c1.append(1)
            else:
                self.category_list_c1.append(0)
        
        self.nb_median1 = MultinomialNB()
        self.nb_median1.fit(self.train_features_c1 , category_list_c1[:])

        self.category_list_c2=[]
        for i in median_list:
            if(i == 2):
                self.category_list_c2.append(1)
            else:
                self.category_list_c2.append(0)
        self.nb_median2 = MultinomialNB()
        self.nb_median2.fit(self.train_features_c1 , self.category_list_c2)

        self.category_list_c3=[]
        for i in median_list:
            if(i == 3):
                self.category_list_c3.append(1)
            else:
                self.category_list_c3.append(0)

        self.nb_median3 = MultinomialNB()
        self.nb_median3.fit(train_features_c1 , self.category_list_c3)       

        self.category_list_c4=[]
        for i in median_list:
            if(i == 4):
                self.category_list_c4.append(1)
            else:
                self.category_list_c4.append(0)

        self.nb_median4 = MultinomialNB()
        self.nb_median4.fit(train_features_c1 , self.category_list_c4[:])
        
        self.category_list_c5=[]
        for i in median_list:
            if(i == 5):
                self.category_list_c5.append(1)
            else:
                self.category_list_c5.append(0)
        self.nb_median5 = MultinomialNB()
        self.nb_median5.fit(train_features_c1 , self.category_list_c5)

        self.category_list_c0=[]
        for i in median_list:
            if(i == 0):
                self.category_list_c0.append(1)
            else:
                self.category_list_c0.append(0)
                
        self.nb_median0 = MultinomialNB()
        self.nb_median0.fit(train_features_c1 , self.category_list_c0)
        
    def __predict_median1_c2(self, path):
        corpus_filtro5_devset = pandas.read_csv(path,encoding='utf-8')
        text_devlist=corpus_filtro5_devset['text']
        pos=0
        for t in text_devlist:
            if type(t) != str:
                text_devlist[pos]="NaN"
            pos+=1
        self.test_features_c1 = vectorizer.transform(text_devlist)
        return self.nb_median1.predict_proba(self.test_features_c1)
    def __predict_median2_c2(self, path):
        corpus_filtro5_devset = pandas.read_csv(path,encoding='utf-8')
        text_devlist=corpus_filtro5_devset['text']
        pos=0
        for t in text_devlist:
            if type(t) != str:
                text_devlist[pos]="NaN"
            pos+=1
        self.test_features_c2 = vectorizer.transform(text_devlist)
        return self.nb_median2.predict_proba(self.test_features_c2)
    def __predict_median3_c2(self, path):
        corpus_filtro5_devset = pandas.read_csv(path,encoding='utf-8')
        text_devlist=corpus_filtro5_devset['text']
        pos=0
        for t in text_devlist:
            if type(t) != str:
                text_devlist[pos]="NaN"
            pos+=1
        self.test_features_c3 = vectorizer.transform(text_devlist)
        return self.nb_median3.predict_proba(self.test_features_c3)
    def __predict_median4_c2(self, path):
        corpus_filtro5_devset = pandas.read_csv(path,encoding='utf-8')
        text_devlist=corpus_filtro5_devset['text']
        pos=0
        for t in text_devlist:
            if type(t) != str:
                text_devlist[pos]="NaN"
            pos+=1
        self.test_features_c4 = vectorizer.transform(text_devlist)
        return self.nb_median4.predict_proba(self.test_features_c4)
    def __predict_median5_c2(self, path):
        corpus_filtro5_devset = pandas.read_csv(path,encoding='utf-8')
        text_devlist=corpus_filtro5_devset['text']
        pos=0
        for t in text_devlist:
            if type(t) != str:
                text_devlist[pos]="NaN"
            pos+=1
        self.test_features_c5 = vectorizer.transform(text_devlist)
        return self.nb_median5.predict_proba(self.test_features_c5)
    def __predict_median0_c2(self, path):
        corpus_filtro5_devset = pandas.read_csv(path,encoding='utf-8')
        text_devlist=corpus_filtro5_devset['text']
        pos=0
        for t in text_devlist:
            if type(t) != str:
                text_devlist[pos]="NaN"
            pos+=1
        self.test_features_c0 = vectorizer.transform(text_devlist)
        return self.nb_median0.predict_proba(self.test_features_c0)
    def __clasiffier_2_train(self):
        corpus_filtro5_trainingset_c2 = pandas.read_csv("corpus_filtro5_trainingset_c2.csv",encoding='utf-8')
        text_list_c2=corpus_filtro5_trainingset_c2['text'][:]
        category_list_c2=corpus_filtro5_trainingset_c2['category'][:]
        pos_c2=0
        for t in text_list_c2:
            if type(t) != str:
                text_list_c2[pos_c2]="NaN"
            pos_c2+=1
        self.train_features_class2 = self.vectorizer.fit_transform(text_list_c2)

        self.nb_2 = MultinomialNB()
        self.nb_2.fit(self.train_features_class2, category_list_c2)

    
    def predict_median(self, path):
        c1_pred=self.__predict_median1_c2(path)
        c2_pred=self.__predict_median2_c2(path)
        c3_pred=self.__predict_median3_c2(path)
        c4_pred=self.__predict_median4_c2(path)
        c5_pred=self.__predict_median5_c2(path)
        c0_pred=self.__predict_median0_c2(path)        
        predicts=[]
        proba=[]
        for i in range( len(c1_pred) ):
            maximum=np.max([c1_pred[i][1],c2_pred[i][1],c3_pred[i][1],c4_pred[i][1],c5_pred[i][1],c0_pred[i][1]])
            if(maximum == c1_pred[i][1]):
                predicts.append(1)
            elif(maximum == c2_pred[i][1]):
                predicts.append(2)
            elif(maximum == c3_pred[i][1]):
                predicts.append(3)
            elif(maximum == c4_pred[i][1]):
                predicts.append(4)
            elif(maximum == c5_pred[i][1]):
                predicts.append(5)
            elif(maximum == c0_pred[i][1]):
                predicts.append(0)
            proba.append(maximum)
        corpus_filtro5_devset_c2 = pandas.read_csv(path,encoding='utf-8')
        text_devlist_c2=corpus_filtro5_devset_c2['text'][:]
        category_devlist_c2=corpus_filtro5_devset_c2['category'][:]
        pos=0
        for t in text_devlist_c2:
            if type(t) != str:
                text_devlist_c2[pos]="NaN"
            pos+=1
        self.test_features_class2 = self.vectorizer.transform(text_devlist_c2)
                    
        return predicts, proba, self.nb_2.predict(self.test_features_class2)

class Clasificador3:
    def __init__(self):
        self.vectorizer = CountVectorizer(stop_words='english')
        self.__clasiffier_3_train()
    def __clasiffier_3_train(self):
        corpus_filtro5_trainingset_c3 = pandas.read_csv("corpus_filtro5_trainingset_c3.csv",encoding='utf-8')
        text_list=corpus_filtro5_trainingset_c3['text'][:]
        category_list=corpus_filtro5_trainingset_c3['category'][:]
        pos=0
        for t in text_list:
            if type(t) != str:
                text_list[pos]="NaN"
            pos+=1
        self.train_features = self.vectorizer.fit_transform(text_list)
        self.nb = MultinomialNB()
        self.nb.fit(self.train_features , category_list)
    def predict_clasiffier_1(self, path):
        corpus_filtro5_devset_c3 = pandas.read_csv(path,encoding='utf-8')
        text_devlist=corpus_filtro5_devset_c3['text'][:]
        category_devlist=corpus_filtro5_devset_c3['category'][:]
        pos=0
        for t in text_devlist:
            if type(t) != str:
                text_devlist[pos]="NaN"
            pos+=1
        self.test_features = self.vectorizer.transform(text_devlist)
        return self.nb.predict(self.test_features)

In [143]:
clasificador = Clasificador1()
print(clasificador.predict_clasiffier_1("corpus_filtro5_devset.csv"))

[1 0 1 0 1 0 0 0 1 0 1 0 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 1 1 1 1 0 0 0 0 0 1
 0 1 0 1 1 0 1 0 1 0 0 0 1 0 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 0
 0 0 1 0 1 1 1 1 0 1 1 1 0 1 1 0 0 1 0 0 0 1 1 1 0 0 1 1 0 1 0 1 1 1 1 0 0
 0 1 1 0 1 1 1 0 1 0 0 1 1 0 0 0 0 0 0 1 1 1 0 1 1 1 0 0 0 1 0 1 1 0 1 1 1
 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 0 1 0 0 0 1 1 0 0 1 1 0 1 1 1 0 0 0 1 0 0 0
 1 1 0 1 0 0 1 1 1 0 1 0 0 1 1 0 1 1 0 1 0 1 1 1 0 0 0 1 0 1 1 1 1 1 0 1 1
 1 0 1 1 1 1 0 0 1 0 1 1 1 0 0 0 0 0 1 1 0 0 0 1 1 1 0 1 0 0 0 1 1 1 1 1 1
 0 1 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 0 0 0 0 1 0 0 0 1 1 1 1 0 1 1 1 0 1 1 1
 1 1 1 1 1 1 0 1 0 0 1 1 0 1 1 0 1 0 1 1 1 0 1 0 0 1 1 1 0 0 0 1 0 1 1 1 0
 1 1 0 1 1 1 0 0 0 1 1 0 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 1 0 0 0 1
 1 1 0 1 0 1 0 0 1 0 0 1 0 1 0 1 1 1 1 0 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 0 1
 1 0 1 1 0 0 1 1 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 1 1 0 1 0 1 0 1 1 1 1 1
 1 1 0 0 1 1 1 1 1 0 0 0 1 1 0 0 1 0 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 0 0 0
 1 1 1 1 0 0 0 0 0 0 0 1 

In [144]:
clasificador=Clasificador2()
predict, proba, humor_prob=clasificador.predict_median("corpus_filtro5_devset_c2.csv")
print(humor_prob)

/home/sebastian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/sebastian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/sebastian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:142: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/sebastian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:152: SettingWithCopyWarning: 
A value i

[1 0 1 0 0 1 1 0 0 0 1 1 0 0 0 1 1 0 1 0 1 1 0 0 1 1 1 1 1 0 1 0 0 0 0 1 0
 1 0 0 0 1 0 1 1 0 1 0 0 0 0 1 0 1 1 0 1 1 1 1 0 0 1 0 0 0 1 0 0 1 1 1 1 0
 1 1 0 1 1 0 1 1 0 0 1 1 1 0 1 1 0 0 1 0 1 1 1 0 0 1 0 1 0 1 1 1 1 0 1 1 1
 0 0 0 1 0 0 1 1 0 1 0 0 1 1 1 0 0 1 1 1 1 0 1 0 1 0 1 1 0 1 1 0 1 1 0 0 0
 1 0 1 0 0 0 1 1 0 1 0 1 0 0 0 1 1 0 1 0 0 1 1 1 0 1 0 0 0 1 1 1 1 1 0 0 1
 0 1 1 0 1 0 1 1 0 1 0 0 1 0 0 1 1 1 0 1 1 1 1 0 0 1 1 1 0 0 0 1 1 0 0 1 1
 0 1 1 0 0 1 1 0 1 0 1 1 0 0 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 0 1
 0 1 1 1 1 0 1 1 0 0 1 0 0 1 0 1 1 0 1 1 0 1 0 1 1 0 0 1 1 0 0 0 1 0 1 0 1
 1 0 0 0 1 0 1 0 1 1 0 0 1 1 1 1 1 0 1 1 0 0 1 0 1 0 0 1 0 1 1 1 0 0 0 1 0
 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 0 0 0 0 0 1 1 0 1 1 1 0 1 1 1 0 1 1 0
 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 1 1 0 0 1 0 0 1 1 1 1 1 1 0 0 1 0 0
 1 0 1 1 1 1 1 1 0 0 1 0 0 1 1 1 0 0 0 0 1 1 0 0 0 1 1 1 0 1 0 0 1 0 0 0 0
 1 1 0 0 1 0 0 1 0 1 0 1 1 1 1 0 0 0 0 1 1 1 1 0 1 1 1 0 1 1 1 0 0 1 0 0 1
 0 1 0 1 0 1 0 0 0 1 1 1 

/home/sebastian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [146]:
clasificador = Clasificador3()
print(clasificador.predict_clasiffier_1("corpus_filtro5_devset_c3.csv"))

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 0
 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 1 1 1 1 1 1 1 